In [1]:
import os
import yaml

import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import tensorflow as tf

from adversarial_generators.fgsm import generate_adversarial_images
from model.VGG19 import VGG19
from preprocess.preprocess import load_data

# Config

In [2]:
with open('config/config.yml', 'r') as stream:
    config = yaml.safe_load(stream)

ROOT_DIRECTORY = os.path.dirname(os.path.abspath('__file__'))

# Load weights to model

In [3]:
input_shape = (config["img_height"], config["img_width"], 3)
model = VGG19(input_shape = input_shape, num_classes = config["num_classes"])
model.load_weights(config["path_to_weights"])

# Compile Model

In [4]:
optimizer = tf.keras.optimizers.Adam(lr = config["learning_rate"], decay = config["learning_rate"] / (config["epochs"]))
loss = config["loss_function"]
metrics = config["metrics"]
model.compile(optimizer = optimizer, loss = loss, metrics = [metrics])

C:\Users\auste\anaconda3\envs\urop\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


# Load data for adversarial training

In [5]:
path_to_set = os.path.join(ROOT_DIRECTORY, config["path_to_data"])
path_to_train_csv = os.path.join(ROOT_DIRECTORY, config["path_to_train_csv"])
path_to_test_csv = os.path.join(ROOT_DIRECTORY, config["path_to_test_csv"])
(X_train, y_train) = load_data(path_to_train_csv, path_to_set, config["img_width"], config["img_height"])
(X_test, y_test) = load_data(path_to_test_csv, path_to_set, config["img_width"], config["img_height"])
X_test, X_adversarial_train, y_test, y_adversarial_train = train_test_split(X_test, y_test, test_size = 0.5, random_state = 0)

loaded: 0
loaded: 500
loaded: 1000
loaded: 1500
loaded: 2000
loaded: 2500
loaded: 3000
loaded: 3500
loaded: 4000
loaded: 4500
loaded: 5000
loaded: 5500
loaded: 6000
loaded: 6500
loaded: 7000
loaded: 7500
loaded: 8000
loaded: 8500
loaded: 9000
loaded: 9500
loaded: 10000
loaded: 10500
loaded: 11000
loaded: 11500
loaded: 12000
loaded: 12500
loaded: 13000
loaded: 13500
loaded: 14000
loaded: 14500
loaded: 15000
loaded: 15500
loaded: 16000
loaded: 16500
loaded: 17000
loaded: 17500
loaded: 18000
loaded: 18500
loaded: 19000
loaded: 19500
loaded: 20000
loaded: 20500
loaded: 21000
loaded: 21500
loaded: 22000
loaded: 22500
loaded: 23000
loaded: 23500
loaded: 24000
loaded: 24500
loaded: 25000
loaded: 25500
loaded: 26000
loaded: 26500
loaded: 27000
loaded: 27500
loaded: 28000
loaded: 28500
loaded: 29000
loaded: 29500
loaded: 30000
loaded: 30500
loaded: 31000
loaded: 31500
loaded: 32000
loaded: 32500
loaded: 33000
loaded: 33500
loaded: 34000
loaded: 34500
loaded: 35000
loaded: 35500
loaded: 36000
lo

# Normalize the data

In [6]:
X_train = X_train.astype("float32") / 255.0
X_adversarial_train = X_adversarial_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

# One-Hot Encode Target value

In [7]:
y_train = tf.keras.utils.to_categorical(y_train, config["num_classes"])
y_adversarial_train = tf.keras.utils.to_categorical(y_adversarial_train, config["num_classes"])
y_test = tf.keras.utils.to_categorical(y_test, config["num_classes"])

# Plot some adversarial images

In [8]:
'''
epsilons = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10]
index = 6122 # Change if you want to see other images 
images = X_test[index:index + 64]
labels = y_test[index:index + 64]
f, axarr = plt.subplots(5,11, figsize = (30, 20))
for i in range(5):
  axarr[i, 0].imshow(images[i])
  axarr[i, 0].set_xlabel("Original class: {}".format(np.argmax(labels, axis = 1)[i]))
for i, eps in enumerate(epsilons):
  adversarial_images = generate_adversarial_images(images, labels, eps, model).numpy()
  new_predictions = model.predict_on_batch(adversarial_images)
  new_predictions = np.argmax(new_predictions, axis = 1)
  for ax in range(5):
    axarr[ax, i + 1].imshow(adversarial_images[ax])
    axarr[ax, i + 1].set_xlabel("New class: {}".format(new_predictions[ax]))
  f.axes[i + 1].set_title('Eps: {}'.format(eps))
plt.show()
'''

'\nepsilons = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10]\nindex = 6122 # Change if you want to see other images \nimages = X_test[index:index + 64]\nlabels = y_test[index:index + 64]\nf, axarr = plt.subplots(5,11, figsize = (30, 20))\nfor i in range(5):\n  axarr[i, 0].imshow(images[i])\n  axarr[i, 0].set_xlabel("Original class: {}".format(np.argmax(labels, axis = 1)[i]))\nfor i, eps in enumerate(epsilons):\n  adversarial_images = generate_adversarial_images(images, labels, eps, model).numpy()\n  new_predictions = model.predict_on_batch(adversarial_images)\n  new_predictions = np.argmax(new_predictions, axis = 1)\n  for ax in range(5):\n    axarr[ax, i + 1].imshow(adversarial_images[ax])\n    axarr[ax, i + 1].set_xlabel("New class: {}".format(new_predictions[ax]))\n  f.axes[i + 1].set_title(\'Eps: {}\'.format(eps))\nplt.show()\n'

# Duplicate model to Model A and Model B

In [9]:
# Model A
model_a = tf.keras.models.clone_model(model)
model_a.load_weights(config["path_to_weights"])
model_a.compile(optimizer = optimizer, loss = loss, metrics = [metrics])

# Model B
model_b = tf.keras.models.clone_model(model)
model_b.load_weights(config["path_to_weights"])
model_b.compile(optimizer = optimizer, loss = loss, metrics = [metrics])

# Accuracy of both models on test set

In [10]:
print('Model A')
model_a.evaluate(X_test, y_test)
print('Model B')
model_b.evaluate(X_test, y_test)

Model A
198/198 [==============================] - 26s 125ms/step - loss: 0.2334 - accuracy: 0.9601
Model B
198/198 [==============================] - 26s 128ms/step - loss: 0.2334 - accuracy: 0.9601


[0.23339110612869263, 0.9600949883460999]

# Adversarial Training

In [11]:
eps = 0.05
adversarial_images = generate_adversarial_images(X_adversarial_train, y_adversarial_train, eps, model_a).numpy()
new_predictions = model.predict_on_batch(adversarial_images)

indexes_of_wrong_images = (np.argmax(new_predictions, axis = 1) != np.argmax(y_adversarial_train, axis = 1))

# Get the images wrongly classified by model A
wrong_classified_images = X_adversarial_train[indexes_of_wrong_images]

# Convert list of prob to one hot encoding for traing model B
new_predictions = new_predictions[indexes_of_wrong_images]
new_predictions = tf.keras.utils.to_categorical(np.argmax(new_predictions, axis = 1))
print(len(new_predictions))

4657


# Create Callback for Early Stopping

In [12]:
callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 3)

# Shuffle train data for adversarial training

In [13]:
# Alpha ratio between train and adv train data
alpha = 5
X_train_new, _, y_train_new, _ = train_test_split(X_train, y_train, train_size = alpha * len(wrong_classified_images), random_state = 0)
X_for_A, y_for_A = sklearn.utils.shuffle(
  np.concatenate((X_train_new, wrong_classified_images)),
  np.concatenate((y_train_new, new_predictions)),
  random_state = 0)
X_for_B, y_for_B = sklearn.utils.shuffle(
  np.concatenate((X_train_new, wrong_classified_images)),
  np.concatenate((y_train_new, y_adversarial_train[indexes_of_wrong_images])),
  random_state = 0)

# Train Model A with incorrect labels

In [14]:
model_a.fit(
  x = X_for_A,
  y = y_for_A,
  batch_size = config["batch_size"],
  verbose = 1,
  validation_split = 0.1,
  callbacks = [callback],
  epochs = config["epochs"])

Epoch 1/20
393/393 [==============================] - 907s 2s/step - loss: 0.9588 - accuracy: 0.8254 - val_loss: 0.8951 - val_accuracy: 0.8290
Epoch 2/20
393/393 [==============================] - 909s 2s/step - loss: 0.8268 - accuracy: 0.8333 - val_loss: 0.8462 - val_accuracy: 0.8329
Epoch 3/20
393/393 [==============================] - 969s 2s/step - loss: 0.7595 - accuracy: 0.8403 - val_loss: 0.7677 - val_accuracy: 0.8426
Epoch 4/20
393/393 [==============================] - 1020s 3s/step - loss: 0.7052 - accuracy: 0.8467 - val_loss: 0.8003 - val_accuracy: 0.8347
Epoch 5/20
393/393 [==============================] - 1024s 3s/step - loss: 0.6443 - accuracy: 0.8545 - val_loss: 0.7331 - val_accuracy: 0.8404
Epoch 6/20
393/393 [==============================] - 1066s 3s/step - loss: 0.6166 - accuracy: 0.8551 - val_loss: 1.4626 - val_accuracy: 0.6501
Epoch 7/20
393/393 [==============================] - 1085s 3s/step - loss: 0.5994 - accuracy: 0.8569 - val_loss: 0.7457 - val_accuracy: 0.

In [15]:
print('Model A')
model_a.evaluate(X_test, y_test)
print('Model B')
model_b.evaluate(X_test, y_test)

Model A
198/198 [==============================] - 22s 112ms/step - loss: 1.8161 - accuracy: 0.6293
Model B
198/198 [==============================] - 24s 123ms/step - loss: 0.2334 - accuracy: 0.9601


[0.23339110612869263, 0.9600949883460999]

# Train Model B with correct labels

In [16]:
model_b.fit(
  x = X_for_B,
  y = y_for_B,
  batch_size = config["batch_size"],
  verbose = 1,
  validation_split = 0.1,
  callbacks = [callback],
  epochs = config["epochs"])

Epoch 1/20
393/393 [==============================] - 1036s 3s/step - loss: 0.3321 - accuracy: 0.9243 - val_loss: 0.1722 - val_accuracy: 0.9488
Epoch 2/20
393/393 [==============================] - 1050s 3s/step - loss: 0.0512 - accuracy: 0.9891 - val_loss: 0.0298 - val_accuracy: 0.9936
Epoch 3/20
393/393 [==============================] - 1071s 3s/step - loss: 0.0400 - accuracy: 0.9912 - val_loss: 0.0222 - val_accuracy: 0.9939
Epoch 4/20
393/393 [==============================] - 958s 2s/step - loss: 0.0215 - accuracy: 0.9951 - val_loss: 0.1651 - val_accuracy: 0.9664
Epoch 5/20
393/393 [==============================] - 901s 2s/step - loss: 0.0265 - accuracy: 0.9943 - val_loss: 0.0397 - val_accuracy: 0.9903
Epoch 6/20
393/393 [==============================] - 794s 2s/step - loss: 0.0197 - accuracy: 0.9961 - val_loss: 0.0259 - val_accuracy: 0.9939


# Save_weights

In [17]:
path_to_weights_a = os.path.join(config["path_to_save_weights"], '{}_weight_a.h5'.format(eps))
path_to_weights_b = os.path.join(config["path_to_save_weights"], '{}_weight_b.h5'.format(eps))
model_a.save_weights(os.path.join(ROOT_DIRECTORY, path_to_weights_a))
model_b.save_weights(os.path.join(ROOT_DIRECTORY, path_to_weights_b))